In [1]:
import lottomain as lm
import importlib as i
i.reload(lm)


# NN
from keras.layers import LSTM, Dense, Bidirectional, Dropout
# optimizer
from tensorflow.keras.optimizers import Adam

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

# normalize
from tensorflow.keras.constraints import min_max_norm

import tensorflow as tf

In [2]:
df = lm.getLotto(link='https://bet.szerencsejatek.hu/cmsfiles/hatos.xls',
                mode="hatos")

In [30]:
# get the x last week
WEEK = 52*30
dfNumpy = df[:WEEK][["1","2","3","4","5","6"]].values
dfNumpy = df[:WEEK][["1","2","3","4","5","6"]]

# Scaler?

In [33]:
#scaler?
#### It has to be done with the original df 
# between -1 and 1
scaler, BATCH, EPOCS = lm.StandardScaler().fit(df.values), 300, 300
# between 0 and 1
scaler, BATCH, EPOCS = lm.MinMaxScaler().fit(df.values), 50, 10

# here I just scaling the last 3 years
transformed_dataset = scaler.transform(dfNumpy.values)
transformed_df = lm.pd.DataFrame(data=transformed_dataset, index=dfNumpy.index)


In [6]:
lm.minMaxValuePrint(transformed_dataset)
chance = lm.minMaxValuePrint(df.values)

 minimum value 0.0, maximum value 0.861111111111111 column 1
 minimum value 0.0, maximum value 0.9444444444444444 column 2
 minimum value 0.05405405405405406, maximum value 0.9189189189189189 column 3
 minimum value 0.05405405405405406, maximum value 0.9729729729729729 column 4
 minimum value 0.08571428571428574, maximum value 1.0 column 5
 minimum value 0.16129032258064518, maximum value 1.0 column 6
 minimum value 1, maximum value 37 column 1
 minimum value 2, maximum value 38 column 2
 minimum value 3, maximum value 40 column 3
 minimum value 6, maximum value 43 column 4
 minimum value 9, maximum value 44 column 5
 minimum value 14, maximum value 45 column 6


In [35]:
def calculate_probabilities(ranges):
    # Számsorok
    numbers = []
    for r in ranges:
        numbers.append(list(range(r[0], r[1]+1)))

    # Lehetséges kombinációk száma
    total_combinations = 1
    for nums in numbers:
        total_combinations *= len(nums)

    # Átfedés kezelése
    for i in range(len(ranges)):
        overlap = set(numbers[i]) & set(numbers[i-1])
        if overlap:
            total_combinations -= 1

    # Esély kiszámítása
    probability = 1 / total_combinations

    return probability

# Számsorok
number_ranges = [
    (1, 37),
    (2, 38),
    (3, 40),
    (6, 43),
    (9, 44),
    (14,45)
]

# Esély kiszámítása
result = calculate_probabilities(number_ranges)
print('{:.30f}'.format(result))

0.000000000439113592550219400607


In [27]:
chance

[32, 36, 37, 37, 38, 38]

In [32]:
t = 1/32*1/36*1/37*1/37*1/38*1/38
t = 1/32

In [34]:
1/t

2277315072.0

In [33]:
print('{:.16f}'.format(t))

0.0000000004391136


In [7]:
scaler.data_max_

array([37., 38., 40., 43., 44., 45.])

In [20]:
#print(scaler.transform([[38,34,32,1,44,45]]))

[[ 1.02777778  0.88888889  0.78378378 -0.13513514  1.          1.        ]]


# Train / Test split

In [7]:
#trainY,trainX = dfNumpy[0:1],dfNumpy[1:WEEK]
trainY,trainX = transformed_df[0:1].values,transformed_df[1:WEEK].values

# Model 1

In [89]:
import numpy as np


# Define the train dataset

# Define the target sequence to predict


# Repeat the target sequence to match the number of samples in the train data
#target_sequence = np.repeat(target_sequence, train_data.shape[0], axis=0)
target_sequence = np.repeat(trainY, trainX.shape[0], axis=0)

# Reshape the train data and target sequence for input to the RNN
train_data = trainX.reshape((trainX.shape[0], trainX.shape[1], 1))
target_sequence = target_sequence.reshape((target_sequence.shape[0], target_sequence.shape[1]))

# Create the RNN model
model = Sequential()
model.add(LSTM(50, input_shape=(train_data.shape[1], train_data.shape[2])))
model.add(Dense(target_sequence.shape[1]))

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(train_data, target_sequence, epochs=100, batch_size=1, verbose=2)

# Make predictions
predictions = model.predict(train_data)

print("Predicted sequence:")
print(predictions)

Epoch 1/100
155/155 - 2s - loss: 0.3099 - 2s/epoch - 12ms/step
Epoch 2/100
155/155 - 0s - loss: 0.0026 - 359ms/epoch - 2ms/step
Epoch 3/100
155/155 - 0s - loss: 8.5341e-04 - 361ms/epoch - 2ms/step
Epoch 4/100
155/155 - 0s - loss: 3.8157e-04 - 355ms/epoch - 2ms/step
Epoch 5/100
155/155 - 0s - loss: 2.1239e-04 - 353ms/epoch - 2ms/step
Epoch 6/100
155/155 - 0s - loss: 1.4567e-04 - 357ms/epoch - 2ms/step
Epoch 7/100
155/155 - 0s - loss: 1.0682e-04 - 355ms/epoch - 2ms/step
Epoch 8/100
155/155 - 0s - loss: 1.0601e-04 - 355ms/epoch - 2ms/step
Epoch 9/100
155/155 - 0s - loss: 1.8244e-04 - 357ms/epoch - 2ms/step
Epoch 10/100
155/155 - 0s - loss: 9.3644e-05 - 355ms/epoch - 2ms/step
Epoch 11/100
155/155 - 0s - loss: 1.1215e-04 - 352ms/epoch - 2ms/step
Epoch 12/100
155/155 - 0s - loss: 1.0392e-04 - 354ms/epoch - 2ms/step
Epoch 13/100
155/155 - 0s - loss: 6.8065e-05 - 368ms/epoch - 2ms/step
Epoch 14/100
155/155 - 0s - loss: 6.8654e-05 - 357ms/epoch - 2ms/step
Epoch 15/100
155/155 - 0s - loss: 8.929

# Model 2

In [31]:
# init 
# Amount of games we need to take into consideration for prediction
window_length = 14
window_length
number_of_features = 6 # how many balls
number_of_rows = 6*52 # based on last 3 years   

X = lm.np.empty([ number_of_rows - window_length, window_length, number_of_features], dtype=float)
y = lm.np.empty([ number_of_rows - window_length, number_of_features], dtype=float)
for i in range(0, number_of_rows-window_length):
    X[i] = transformed_df.iloc[i : i+window_length, 0 : number_of_features]
    y[i] = transformed_df.iloc[i+window_length : i+window_length+1, 0 : number_of_features]
    


ValueError: could not broadcast input array from shape (0,6) into shape (6,)

In [37]:
import numpy as np

window_length = 28
number_of_features = 6
number_of_rows = 6 * 52

X = np.empty((number_of_rows - window_length, window_length, number_of_features), dtype=float)
y = np.empty((number_of_rows - window_length, number_of_features), dtype=float)

for i in range(0, number_of_rows - window_length):
    try:
        X[i] = transformed_df.iloc[i : i + window_length, 0 : number_of_features]
        y[i] = transformed_df.iloc[i + window_length : i + window_length + 1, 0 : number_of_features].values.reshape(number_of_features,)
    except (ValueError, IndexError):
        print(f"Error encountered at index {i + window_length}. Skipping...")

In [53]:
model = Sequential()
parameters = 240
model.add(Bidirectional(LSTM(parameters, input_shape = (window_length, number_of_features), return_sequences = True)))
# reduce overfitting
model.add(Dropout(0.2))
# second layer
model.add(Bidirectional(LSTM(parameters, input_shape = (window_length, number_of_features), return_sequences = True)))
# second dropout
model.add(Dropout(0.2))

model.add(Bidirectional(LSTM(parameters, input_shape = (window_length, number_of_features), return_sequences = True)))

model.add(Bidirectional(LSTM(parameters, input_shape = (window_length, number_of_features), return_sequences = False)))

# add Dense layer
model.add(Dense(59))

# output
model.add(Dense(number_of_features))

# complier
model.compile(optimizer=Adam(learning_rate=0.0001), loss ='mse', metrics=['accuracy'])

# Model 2 with Output constrains

In [43]:
# Set output constraints
min_value = 0.0
max_value = 1.0

model = Sequential()
parameters = 240
model.add(Bidirectional(LSTM(parameters, input_shape = (window_length, number_of_features), return_sequences = True)))
# reduce overfitting
model.add(Dropout(0.2))
# second layer
model.add(Bidirectional(LSTM(parameters, input_shape = (window_length, number_of_features), return_sequences = True)))
# second dropout
model.add(Dropout(0.2))

model.add(Bidirectional(LSTM(parameters, input_shape = (window_length, number_of_features), return_sequences = True)))

model.add(Bidirectional(LSTM(parameters, input_shape = (window_length, number_of_features), return_sequences = False)))

# Define the output constraints as a custom constraint
output_constraint = min_max_norm(min_value=min_value, max_value=max_value)

# add Dense layer
model.add(Dense(30, activation='relu'))

# output
model.add(Dense(number_of_features, activation='sigmoid'))

# Apply the constraint to the Dense layer
model.layers[-1].kernel_constraint = output_constraint
model.layers[-1].bias_constraint = output_constraint

model.layers[-2].kernel_constraint = output_constraint
model.layers[-2].bias_constraint = output_constraint

# complier
# New model complier
def sequence_accuracy(y_true, y_pred):
    # Convert the predicted and true sequences to tensors
    y_true = tf.convert_to_tensor(y_true)
    y_pred = tf.convert_to_tensor(y_pred)

    # Compute element-wise matches
    matches = tf.cast(tf.equal(y_true, y_pred), tf.float32)
    #print(matches)

    # Calculate the accuracy for each element in the sequence
    per_element_accuracy = tf.reduce_mean(matches, axis=-1)
    #print(per_element_accuracy)

    # Calculate the overall accuracy
    accuracy = tf.reduce_mean(per_element_accuracy)
    #print(accuracy)

    return accuracy

model.compile(optimizer=Adam(learning_rate=0.0001), loss='mse', metrics=["accuracy", sequence_accuracy])

In [41]:
tX = [[1,1,1,1,1,1],[10,10,10,10,10,10]]
ty = [[2,3,4,5,6,7],[7,8,9,10,11,12]]

In [42]:
history = model.fit(x=tX, y=ty, batch_size=BATCH, epochs=EPOCS, verbose=2)

Epoch 1/20


ValueError: in user code:

    File "C:\Users\MUSIC\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\MUSIC\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\MUSIC\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\MUSIC\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\MUSIC\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\MUSIC

# FIT

In [44]:
EPOCS = 20
BATCH = 100
# fit it
history = model.fit(x=X, y=y, batch_size=BATCH, epochs=EPOCS, verbose=2)

Epoch 1/20
3/3 - 20s - loss: 0.0846 - accuracy: 0.8239 - sequence_accuracy: 0.0000e+00 - 20s/epoch - 7s/step
Epoch 2/20
3/3 - 5s - loss: 0.0756 - accuracy: 0.8345 - sequence_accuracy: 0.0000e+00 - 5s/epoch - 2s/step
Epoch 3/20
3/3 - 4s - loss: 0.0638 - accuracy: 0.8345 - sequence_accuracy: 0.0000e+00 - 4s/epoch - 1s/step
Epoch 4/20
3/3 - 4s - loss: 0.0521 - accuracy: 0.8345 - sequence_accuracy: 0.0000e+00 - 4s/epoch - 1s/step
Epoch 5/20
3/3 - 4s - loss: 0.0442 - accuracy: 0.8345 - sequence_accuracy: 0.0000e+00 - 4s/epoch - 1s/step
Epoch 6/20
3/3 - 4s - loss: 0.0407 - accuracy: 0.8345 - sequence_accuracy: 0.0000e+00 - 4s/epoch - 1s/step
Epoch 7/20
3/3 - 5s - loss: 0.0371 - accuracy: 0.8345 - sequence_accuracy: 0.0000e+00 - 5s/epoch - 2s/step
Epoch 8/20
3/3 - 4s - loss: 0.0341 - accuracy: 0.8345 - sequence_accuracy: 0.0000e+00 - 4s/epoch - 1s/step
Epoch 9/20
3/3 - 4s - loss: 0.0336 - accuracy: 0.8345 - sequence_accuracy: 0.0000e+00 - 4s/epoch - 1s/step
Epoch 10/20
3/3 - 5s - loss: 0.0334

In [46]:
# the model is done now. Lets take the last X games and try to predict
BASEDON = 10
#to_pred = df[1:BASEDON][["1","2","3","4","5","6"]].values
# predict based on the last x weeks
to_pred = df[0:BASEDON][["1","2","3","4","5","6"]].values
# scaling
to_pred = scaler.transform(to_pred)
pred = df[0:1][["1","2","3","4","5","6"]].values

y_pred = model.predict(lm.np.array([to_pred]))
print("The predicted numbers in the last lottery game are:", scaler.inverse_transform(y_pred).astype(int)[0])

1/1 [==============================] - 0s 31ms/step
The predicted numbers in the last lottery game are: [11 16 20 25 29 34]


In [181]:
y_pred

array([[0.03890081, 0.2079196 , 0.39232567, 0.5402744 , 0.7727668 ,
        0.92842317]], dtype=float32)

In [20]:
df[0:BASEDON][["1","2","3","4","5","6"]].values

array([[ 5, 18, 20, 26, 35, 43],
       [ 3,  8, 12, 26, 34, 37],
       [ 2, 26, 27, 34, 36, 37],
       [ 2,  3, 17, 19, 36, 38],
       [ 8, 20, 25, 28, 35, 45],
       [11, 14, 21, 22, 34, 40],
       [ 3,  4,  8, 21, 22, 29],
       [ 2,  9, 10, 19, 22, 26],
       [10, 13, 15, 31, 36, 42],
       [ 4, 14, 19, 23, 27, 42],
       [19, 28, 31, 33, 37, 43],
       [ 6, 23, 26, 27, 34, 37],
       [17, 23, 35, 36, 37, 43],
       [ 9, 18, 25, 29, 33, 40],
       [ 4,  7, 16, 26, 28, 37],
       [ 1,  4, 14, 19, 25, 31],
       [13, 14, 16, 22, 27, 33],
       [17, 18, 30, 32, 34, 35],
       [10, 12, 14, 29, 36, 37],
       [ 3,  7, 15, 19, 32, 40],
       [ 5, 18, 21, 26, 37, 44],
       [13, 20, 24, 26, 38, 42],
       [ 1,  3, 17, 26, 40, 45],
       [ 2, 11, 12, 17, 37, 44],
       [ 3,  5,  6, 14, 35, 37],
       [ 6,  9, 16, 19, 23, 29],
       [ 4, 14, 19, 28, 43, 44],
       [16, 18, 28, 36, 41, 45],
       [ 1, 13, 24, 26, 27, 31],
       [ 1,  4, 26, 32, 36, 44],
       [ 6

In [144]:
pred

array([[ 2, 26, 27, 34, 36, 37]], dtype=int64)